## Create PLY File

In [2]:
import numpy
vertex = numpy.array([(0, 0),
                      (0, 1),
                      (1, 0),
                      (1, 1)],
                      dtype=[('x', 'f4'), ('y', 'f4')])

face = numpy.array([([0, 1, 2, 3], 255, 255, 255)],
                    dtype=[('vertex_indices', 'i4', (4,)),
                           ('red', 'u1'), ('green', 'u1'),
                           ('blue', 'u1')])

In [8]:
from plyfile import PlyData, PlyElement
v = PlyElement.describe(vertex, 'vertex_list')
f = PlyElement.describe(face, 'face_list')
PlyData([v, f], text = True).write('square_2d.ply')

## Read  PLY File

In [16]:
plydata = PlyData.read('square_2d.ply')

In [26]:
plydata.elements[0].data

array([( 0.,  0.), ( 0.,  1.), ( 1.,  0.), ( 1.,  1.)],
      dtype=[('x', '<f4'), ('y', '<f4')])

In [27]:
plydata.elements[1].data

array([(array([0, 1, 2, 3]), 255, 255, 255)],
      dtype=[('vertex_indices', 'O'), ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')])

## DCEL

In [60]:
class Vertex():
    def __init__(self, coordinate = (0, 0), color = None):
        self.x = coordinate[0]
        self.y = coordinate[1]
    
    def set_incidence_edge(self, halfedge):
        self.incidence_edge = halfedge
        
class Face():
    def __init__(self, outer_component = None, inner_component = []):
        self.outer_component = outer_component
        self.inner_component = inner_component
        
    def add_inner_component(self, halfedge):
        self.inner_component.append(halfedge)
        
class HalfEdge():
    def __init__(self, origin, has_twin = False):
        self.origin = origin
        self.has_twin = has_twin
        
    def set_incidence_face(self, incidence_face):
        self.incidence_face = incidence_face
        
    def set_twin(self, twin_edge):
        self.twin = twin_edge
        self.has_twin = True
        
    def set_next(self, next_edge):
        self.next = next_edge
        
    def set_prev(self, prev_edge):
        self.prev = prev_edge
    

In [61]:
class DCEL():
    def __init__(self):
        self.vertex_list = []
        self.face_list = []
        self.half_edge_list = []
        
    def __construct_from_plyfile_without_twin__(self, plydata):
        for coordinate in plydata.elements[0].data:
            vertex = Vertex(coordinate)
            self.vertex_list.append(vertex)
            
        for f in plydata.elements[1].data:
            # create edge list constructing a face
            face = f[0]
            edges = []
            half_edges = []
            for i in range(len(face)):
                if i < len(face)-1:
                    edges.append((face[i], face[i+1]))
                else:
                    edges.append((face[i], face[0]))
            print(edges)
            
            for edge in edges:
                print(edge[0])
                half_edge = HalfEdge(self.vertex_list[edge[0]])
                self.vertex_list[edge[0]].set_incidence_edge(half_edge)
                half_edges.append(half_edge)
                
            face = Face(half_edges[0])
            self.face_list.append(face)
            
            # set next and prev pointer of half-edges in half_edges
            for i in range(len(half_edges)):
                
                if i == 0:
                    half_edges[i].set_next(half_edges[i+1])
                    half_edges[i].set_prev(half_edges[-1])
                    half_edges[i].set_incidence_face(face)
                elif i == len(half_edges)-1:
                    half_edges[i].set_next(half_edges[0])
                    half_edges[i].set_prev(half_edges[i-1])
                    half_edges[i].set_incidence_face(face)
                else:
                    half_edges[i].set_next(half_edges[i+1])
                    half_edges[i].set_prev(half_edges[i-1])
                    half_edges[i].set_incidence_face(face)
                    
            self.half_edge_list.append(half_edges)
            
    def construct_from_plyfile(self, plydata):
        self.__construct_from_plyfile_without_twin__(plydata)

In [62]:
la = DCEL()

In [63]:
la.construct_from_plyfile_without_twin(plydata)

[(0, 1), (1, 2), (2, 3), (3, 0)]
0
1
2
3


In [74]:
la.vertex_list[3].incidence_edge

In [76]:
la.face_list[0].outer_component

In [78]:
la.half_edge_list

[[<__main__.HalfEdge at 0x2adfebcfef0>,

In [91]:
la.half_edge_list[0][3].prev